In [15]:
!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard

ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview
You should consider upgrading via the '/raid/home/aayush/git/fitapp_analysis/fitapp_aayush/bin/python -m pip install --upgrade pip' command.


In [17]:
import functools
from gym import spaces
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# RecSim imports
from recsim import agent
from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym
from recsim.simulator import runner_lib


In [3]:
from recsim.environments import interest_exploration

In [4]:
env_config = {'slate_size': 2,
              'seed': 0,
              'num_candidates': 15,
              'resample_documents': True}
ie_environment = interest_exploration.create_environment(env_config)

In [5]:
initial_observation = ie_environment.reset()

In [6]:
print('User Observable Features')
print(initial_observation['user'])
print('User Response')
print(initial_observation['response'])
print('Document Observable Features')
for doc_id, doc_features in initial_observation['doc'].items():
  print('ID:', doc_id, 'features:', doc_features)

User Observable Features
[]
User Response
None
Document Observable Features
ID: 15 features: {'quality': array(1.22720163), 'cluster_id': 1}
ID: 16 features: {'quality': array(1.29258489), 'cluster_id': 1}
ID: 17 features: {'quality': array(1.23977078), 'cluster_id': 1}
ID: 18 features: {'quality': array(1.46045555), 'cluster_id': 1}
ID: 19 features: {'quality': array(2.10233425), 'cluster_id': 0}
ID: 20 features: {'quality': array(1.09572905), 'cluster_id': 1}
ID: 21 features: {'quality': array(2.37256963), 'cluster_id': 0}
ID: 22 features: {'quality': array(1.34928002), 'cluster_id': 1}
ID: 23 features: {'quality': array(1.00670188), 'cluster_id': 1}
ID: 24 features: {'quality': array(1.20448562), 'cluster_id': 1}
ID: 25 features: {'quality': array(2.18351159), 'cluster_id': 0}
ID: 26 features: {'quality': array(1.19411585), 'cluster_id': 1}
ID: 27 features: {'quality': array(1.03514646), 'cluster_id': 1}
ID: 28 features: {'quality': array(2.29592623), 'cluster_id': 0}
ID: 29 feature

In [7]:
print('Document observation space')
for key, space in ie_environment.observation_space['doc'].spaces.items():
  print(key, ':', space)
print('Response observation space')
print(ie_environment.observation_space['response'])
print('User observation space')
print(ie_environment.observation_space['user'])


Document observation space
15 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
16 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
17 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
18 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
19 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
20 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
21 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
22 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
23 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
24 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
25 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
26 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
27 : Dict(cluster_id: Discrete(2), quality: Box(0.0, inf, (), float32))
28 : Dict(cluster_id: Discrete(2), qu

In [8]:
slate = [0, 1]
for slate_doc in slate:
  print(list(initial_observation['doc'].items())[slate_doc])

('15', {'quality': array(1.22720163), 'cluster_id': 1})
('16', {'quality': array(1.29258489), 'cluster_id': 1})


In [9]:
ie_environment.action_space

MultiDiscrete([15 15])

In [10]:
observation, reward, done, _ = ie_environment.step(slate)

In [11]:
from recsim.agent import AbstractEpisodicRecommenderAgent

In [12]:
class StaticAgent(AbstractEpisodicRecommenderAgent):
  def __init__(self, observation_space, action_space):
    # Check if document corpus is large enough.
    if len(observation_space['doc'].spaces) < len(action_space.nvec):
      raise RuntimeError('Slate size larger than size of the corpus.')
    super(StaticAgent, self).__init__(action_space)

  def step(self, reward, observation):
    print(observation)
    return list(range(self._slate_size))

In [19]:
def create_agent(sess, environment, eval_mode, summary_writer=None):
  return StaticAgent(environment.observation_space, environment.action_space)
tf.compat.v1.disable_eager_execution()

tmp_base_dir = '/tmp/recsim/'

runner = runner_lib.EvalRunner(
  base_dir=tmp_base_dir,
  create_agent_fn=create_agent,
  env=ie_environment,
  max_eval_episodes=1,
  max_steps_per_episode=5,
  test_mode=True)

INFO:tensorflow:max_eval_episodes = 1


INFO:tensorflow:max_eval_episodes = 1


INFO:tensorflow:max_steps_per_episode = 5


INFO:tensorflow:max_steps_per_episode = 5
2022-10-04 13:01:05.571567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 13:01:06.445684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /raid/home/aayush/git/fitapp_analysis/fitapp_aayush/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-04 13:01:06.445790: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /raid/home/aayush/git/fitap

In [20]:
from recsim.agents.layers.cluster_click_statistics import ClusterClickStatsLayer

In [21]:
static_agent = StaticAgent(ie_environment.observation_space,
                           ie_environment.action_space)
static_agent.step(reward, observation)

{'user': array([], dtype=float64), 'doc': OrderedDict([('30', {'quality': array(2.48922445), 'cluster_id': 0}), ('31', {'quality': array(2.12592661), 'cluster_id': 0}), ('32', {'quality': array(1.27448139), 'cluster_id': 1}), ('33', {'quality': array(1.21799112), 'cluster_id': 1}), ('34', {'quality': array(1.17770375), 'cluster_id': 1}), ('35', {'quality': array(2.07948915), 'cluster_id': 0}), ('36', {'quality': array(1.14167652), 'cluster_id': 1}), ('37', {'quality': array(1.20529165), 'cluster_id': 1}), ('38', {'quality': array(1.2424684), 'cluster_id': 1}), ('39', {'quality': array(1.87279668), 'cluster_id': 0}), ('40', {'quality': array(1.19644888), 'cluster_id': 1}), ('41', {'quality': array(1.28254021), 'cluster_id': 1}), ('42', {'quality': array(2.01558539), 'cluster_id': 0}), ('43', {'quality': array(2.46400483), 'cluster_id': 0}), ('44', {'quality': array(1.33980633), 'cluster_id': 1})]), 'response': ({'click': 0, 'quality': array(1.22720163), 'cluster_id': 1}, {'click': 0, 'q

[0, 1]

In [22]:
cluster_static_agent = ClusterClickStatsLayer(StaticAgent,
                                              ie_environment.observation_space,
                                              ie_environment.action_space)
cluster_static_agent.step(reward, observation)

{'user': {'raw_observation': array([], dtype=float64), 'sufficient_statistics': {'impression_count': array([0, 2]), 'click_count': array([0, 0])}}, 'doc': OrderedDict([('30', {'quality': array(2.48922445), 'cluster_id': 0}), ('31', {'quality': array(2.12592661), 'cluster_id': 0}), ('32', {'quality': array(1.27448139), 'cluster_id': 1}), ('33', {'quality': array(1.21799112), 'cluster_id': 1}), ('34', {'quality': array(1.17770375), 'cluster_id': 1}), ('35', {'quality': array(2.07948915), 'cluster_id': 0}), ('36', {'quality': array(1.14167652), 'cluster_id': 1}), ('37', {'quality': array(1.20529165), 'cluster_id': 1}), ('38', {'quality': array(1.2424684), 'cluster_id': 1}), ('39', {'quality': array(1.87279668), 'cluster_id': 0}), ('40', {'quality': array(1.19644888), 'cluster_id': 1}), ('41', {'quality': array(1.28254021), 'cluster_id': 1}), ('42', {'quality': array(2.01558539), 'cluster_id': 0}), ('43', {'quality': array(2.46400483), 'cluster_id': 0}), ('44', {'quality': array(1.33980633

[0, 1]

In [23]:
from recsim.agents.layers.abstract_click_bandit import AbstractClickBanditLayer

In [24]:
class GreedyClusterAgent(agent.AbstractEpisodicRecommenderAgent):
  """Simple agent sorting all documents of a topic according to quality."""

  def __init__(self, observation_space, action_space, cluster_id, **kwargs):
    del observation_space
    super(GreedyClusterAgent, self).__init__(action_space)
    self._cluster_id = cluster_id

  def step(self, reward, observation):
    del reward
    my_docs = []
    my_doc_quality = []
    for i, doc in enumerate(observation['doc'].values()):
      if doc['cluster_id'] == self._cluster_id:
        my_docs.append(i)
        my_doc_quality.append(doc['quality'])
    if not bool(my_docs):
      return []
    sorted_indices = np.argsort(my_doc_quality)[::-1]
    return list(np.array(my_docs)[sorted_indices])

In [26]:
num_topics = list(ie_environment.observation_space.spaces['doc']
                    .spaces.values())[0].spaces['cluster_id'].n
base_agent_ctors = [
      functools.partial(GreedyClusterAgent, cluster_id=i)
      for i in range(num_topics)
  ]

In [27]:
bandit_ctor = functools.partial(AbstractClickBanditLayer,
                                arm_base_agent_ctors=base_agent_ctors)
cluster_bandit = ClusterClickStatsLayer(bandit_ctor,
                                        ie_environment.observation_space,
                                        ie_environment.action_space)

In [28]:
observation0 = ie_environment.reset()
slate = cluster_bandit.begin_episode(observation0)
print("Cluster bandit slate 0:")
doc_list = list(observation0['doc'].values())
for doc_position in slate:
  print(doc_list[doc_position])

Cluster bandit slate 0:
{'quality': array(1.46868751), 'cluster_id': 1}
{'quality': array(1.42269182), 'cluster_id': 1}
